In [4]:
!pip install gosdt
#!pip3 install attrs packaging editables pandas sklearn sortedcontainers gmpy2 matplotlib
#!pip3 install auto-sklearn
#!pip install tpot
!pip install gplearn
#!pip install autogluon
#!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.0/749.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.8 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=ab9213c61044bd56d8428faa5ab452bfdc54a9eb50fac327eec6f52a3c5846e8
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
#import gosdt
#import autosklearn.regression
#from tpot import TPOTRegressor
from gplearn.genetic import SymbolicRegressor
import xgboost as xgb
#from autogluon.tabular import TabularDataset, TabularPredictor

In [6]:
META_DATASET_FIT = "meta_dataset_fit.csv"
META_DATASET_TRAIN = "meta_dataset_train.csv"
META_DATASET_TEST = "meta_dataset_test.csv"
RESULTS_FOLDER_NAME = "results"
DATA_FOLDER_NAME = "data"
current_folder = globals()['_dh'][0]
RESULTS_FOLDER_PATH = os.path.join(current_folder, RESULTS_FOLDER_NAME)
DATA_FOLDER_PATH = os.path.join(current_folder, DATA_FOLDER_NAME)
CONFIG_FILE_PATH = os.path.join(current_folder, DATA_FOLDER_NAME, "config.csv")
RANDOM_STATE = 73  # Shaldon number is fund

In [7]:
def metric(y_pred, y):
    metricList = []
    metricList.append(mean_absolute_error(y_true=y,y_pred=y_pred))
    metricList.append(mean_squared_error(y_true=y,y_pred=y_pred))
    metricList.append(r2_score(y_true=y,y_pred=y_pred))

    return metricList

In [8]:
from google.colab import files
files.upload()


Saving data.zip to data.zip


In [9]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/2012_Industry_Data_by_Industry_and_StatenoNa.csv  
  inflating: data/2012_Retail_Sales_by_Store_Type_within_Product_Category.csv  
  inflating: data/2016_all_counties_simp.csv  
  inflating: data/Big_Mac_Index_transposed.csv  
  inflating: data/Brent_Oil_Futures_Historical_DatanoNa.csv  
  inflating: data/BSE_30_ADANIPORTSnoNa.csv  
  inflating: data/BSE_30_ASIANPAINTnoNa.csv  
  inflating: data/BSE_30_AXISBANKnoNa.csv  
  inflating: data/BSE_30_BAJAJ-AUTOnoNa.csv  
  inflating: data/BSE_30_BHARTIARTLnoNa.csv  
  inflating: data/BSE_30_COALINDIAnoNa.csv  
  inflating: data/BSE_30_DRREDDYnoNa.csv  
  inflating: data/BSE_30_HDFCBANKnoNa.csv  
  inflating: data/BSE_30_HDFCnoNa.csv  
  inflating: data/BSE_30_HEROMOTOCOnoNa.csv  
  inflating: data/BSE_30_HINDUNILVRnoNa.csv  
  inflating: data/BSE_30_ICICIBANKnoNa.csv  
  inflating: data/BSE_30_INDUSINDBKnoNa.csv  
  inflating: data/BSE_30_INFYnoNa.csv  
  inflating: data/BSE_30_ITCnoNa

In [10]:
create_paths = [os.path.join(current_folder, RESULTS_FOLDER_PATH),
                        os.path.join(current_folder, DATA_FOLDER_PATH)]
for path in create_paths:
  try:
    os.mkdir(path)
  except Exception as error:
    pass

In [11]:
columns = ["LR","GPlearn","XGBOOST"]

In [12]:
index = []
fit_performance = []
train_performance = []
test_performance = []

config_df = {row["file_name"]: {"is_time_series": int(row["is_time_series"]),
                                        "window": int(row["window"]),
                                        "y_target_col": row["y_target_col"]}
                     for row_index, row in pd.read_csv(CONFIG_FILE_PATH).iterrows()}

In [13]:
def _to_time_series(data: pd.DataFrame,
                    n_in: int,
                    n_out: int = 1,
                    dropnan: bool = True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Sequence of observations as a list or NumPy array.
    n_in: Number of lag observations as input (X).
    n_out: Number of observations as output (y).
    dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    if not isinstance(data, pd.DataFrame):
        df = pd.DataFrame(data)
    else:
        df = data
    cols = []
    names = []
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols,
                    axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [14]:
for data_path in config_df: #glob(os.path.join(DATA_FOLDER_PATH, "*.csv")):

  # if we have this dataset, just skip it
  root_path = "data/"
  file_ext = ".csv"
  file_path = root_path + data_path + file_ext
  file_name = os.path.basename(data_path.split(".csv")[0])
  print(file_name)
  if file_name in index:
      continue

  # load dataset
  df = pd.read_csv(file_path, thousands=",")
  # check if we need to make from time_series
  if config_df[file_name]["is_time_series"]:
      answer = []
      for row, row_index in df.iterrows():
          ts_row = _to_time_series(list(row_index), 4)
          answer.append(ts_row)
      df = pd.concat(answer)
      #df = Main._to_time_series(data=df, n_in=config_df[file_name]["window"])

  # make sure everything in the dataset is legit
  df.dropna(inplace=True)
  for col in list(df):
      df[col] = pd.to_numeric(df[col], errors='coerce')
  df.reset_index(inplace=True, drop=True)

  # prepare where to save results
  row_fit = []
  row_train = []
  row_test = []

  # put the target column in the start
  #y = df[config_df[file_name]["y_target_col"]]
  df.insert(0,
            config_df[file_name]["y_target_col"],
            df.pop(config_df[file_name]["y_target_col"]))
  #x = df.drop(config_df[file_name]["y_target_col"])
  # split to x and y
  x = df.iloc[:, 0:]
  #copy_x = x.copy()
  y = df.iloc[:, 0]
  #copy_y = y.copy()

  # split to train and test for later
  x_train, x_test, y_train, y_test = train_test_split(x,
                                                      y,
                                                      test_size=0.2,
                                                      random_state=RANDOM_STATE)
  # train models #
  # TODO: david - add logic for all the models here
  for model in [LinearRegression(),SymbolicRegressor(),xgb.XGBRegressor()]:
      #run model on all data
      fit_model = model
      fit_model.fit(x, y)
      y_pred = fit_model.predict(x)
      row_fit.append(metric(y_pred=y_pred, y=y))
      #run model on train
      real_model = model
      real_model.fit(x_train, y_train)
      y_pred_train = fit_model.predict(x_train)
      row_train.append(metric(y_pred=y_pred_train, y=y_train))
      #run model on test
      y_pred_test = fit_model.predict(x_test)
      row_test.append(metric(y_pred=y_pred_test, y=y_test))

  # store results for this dataset
  fit_performance.append(row_fit)
  train_performance.append(row_train)
  test_performance.append(row_test)

  # recall the file name so we will use it later, making sure the order is not important
  index.append(file_name)

  # save all three datasets of the answers - this is not optimal but helps if something clashes
  pd.DataFrame(data=fit_performance,
                index=index,
                columns=columns).to_csv(os.path.join(RESULTS_FOLDER_PATH, META_DATASET_FIT))
  pd.DataFrame(data=train_performance,
                index=index,
                columns=columns).to_csv(os.path.join(RESULTS_FOLDER_PATH, META_DATASET_TRAIN))
  pd.DataFrame(data=test_performance,
                index=index,
                columns=columns).to_csv(os.path.join(RESULTS_FOLDER_PATH, META_DATASET_TEST))

2012_Industry_Data_by_Industry_and_StatenoNa
2012_Retail_Sales_by_Store_Type_within_Product_Category
2016_all_counties_simp
Big_Mac_Index_transposed
Brent_Oil_Futures_Historical_DatanoNa
BSE_30_ADANIPORTSnoNa
BSE_30_ASIANPAINTnoNa
BSE_30_AXISBANKnoNa
BSE_30_BAJAJ-AUTOnoNa
BSE_30_BHARTIARTLnoNa
BSE_30_COALINDIAnoNa
BSE_30_DRREDDYnoNa
BSE_30_HDFCBANKnoNa
BSE_30_HDFCnoNa
BSE_30_HEROMOTOCOnoNa
BSE_30_HINDUNILVRnoNa
BSE_30_ICICIBANKnoNa
BSE_30_INDUSINDBKnoNa
BSE_30_INFYnoNa
BSE_30_ITCnoNa
BSE_30_KOTAKBANKnoNa
BSE_30_LTnoNa
BSE_30_M&MnoNa
BSE_30_MARUTInoNa
BSE_30_ONGCnoNa
BSE_30_POWERGRIDnoNa
BSE_30_RELIANCEnoNa
BSE_30_SBINnoNa
BSE_30_SUNPHARMAnoNa
BSE_30_TATAMOTORSnoNa
BSE_30_TATAMTRDVRnoNa
BSE_30_TATASTEELnoNa
BSE_30_TCSnoNa
BSE_30_WIPROnoNa
BSE_30_YESBANKnoNa
crecimiento_economiconoNa
Crude_Oil_WTI_Futures_Historical_DatanoNa
crypto_BNB_USDnoNa
crypto_Cardano_USDnoNa
crypto_Dogecoin_USDnoNa
crypto_Ethereum_USDnoNa
crypto_Litecoin_USDnoNa
crypto_Polkadot_USDnoNa
crypto_TRON_USDnoNa
crypto_

In [15]:
!zip -r ./results.zip ./results/
files.download('./results.zip')

  adding: results/ (stored 0%)
  adding: results/meta_dataset_test.csv (deflated 56%)
  adding: results/meta_dataset_train.csv (deflated 57%)
  adding: results/meta_dataset_fit.csv (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>